# 2024 삼성 무선사업부 생성형 AI 실습 과제
#### PDF 문서 기반 질의응답 - 97158424 좌경윤

PDF 문서를 기반으로 질의응답 기능을 구현한다.

In [22]:
# 1. 패키지 설치

%pip install chromadb
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
# 2. 기본 패키지 설정 & AzureOpenAI 환경설정

# 환경 변수 설정!! 
import os
import langchain

from config_azure import (
    AZURE_OPENAI_API_VERSION,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY
)
# lagchain 사용시 반드시 환경변수 값 사용!!
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [24]:
#3 pdf 문서 읽어 오기
# [제공 문서]
# 2024GenAI_Mineral.pdf

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("2024GenAI_Mineral.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='- 1 -\n광부 보도자료\n보도시점 2024. 2. 20.(화) 12:00 배포2024. 2. 20.(화) 08:00\n10대 ‧ 20대 절반 이상이 혈중 엽산 부족\n- 10대와 20대에서 , 여자보다 남자에서 , 혈중 엽산 결핍 비율이 높아\n  *(혈중 엽산 부족 비율) 10대 남자 63.5%( 여자 51.2%), 20대 남자 71.3%( 여자 46.0%)\n- 엽산 섭취량은 권장섭취량의 약 77%(‘22년 기준), 20대에서 가장 낮아 \n-심혈관질환 등 만성질환의 예방‧관리를 위해 적정 혈중 엽산 농도 유지 필요\n  질병관리청 (청장 지영미 )은 “우리나라 청소년과 젊은 성인의 절반 이상에서 \n혈중 엽산이 적정 수준에 미치지 못해 부족 상태”라는 연구 결과를 전문 학술지에 \n발표하였다 .\n  엽산(또는 비타민 B9)은 세포성장과 분열에 중요한 역할을 하는 수용성 \n비타민 으로, 태아의 성장 발달을 위해 임신 전 및 임신·수유기 여성이 충분히 \n섭취해야 하는 영양소 로 잘 알려져 있다.\n  또한 성인에서의 혈중 엽산 결핍은 심혈관질환 등 만성질환 발생과 연관성 이 \n있는 것으로 보고되고 있다.\n  국립보건연구원은 「국민건강영양조사 제6기(2013-2015) 」 자료를 활용하여 \n우리나라 10세 이상 남녀 8,016 명의 혈중 엽산, 비타민 B12* 및 호모시스테인** \n농도를 비교 분석하였다 .\n*비타민B12:세포분열과신경계기능에역할을하는비타민으로,동물성식품섭취\n부족및노화로인한흡수불량으로결핍될수있음.\n**호모시스테인:엽산을포함한비타민B군섭취부족시증가하는황-함유아미노산으로\n높은호모시스테인농도는동맥의손상과혈관의혈전을유발할수있음.\n<혈중엽산및비타민B12상태분석기준치>\n-(혈중엽산)세계보건기구(WHO)기준*에따라6.8nmol/L미만일경우결핍,\n6.8-13.4nmol/L미만일경우경계결핍으로분류하여분석\n*큰적혈구빈혈발생기준으로제안한기준치\n-(비타민B12)미국국립의학원(USNationa

In [25]:
# 4. Vetor Store에 텍스트 벡터값 저장(검색가능함)

# Vector store는 ChromaDB 또는 FAISS 를 사용한다.
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma

# Embedding 모델은 text-embedding-ada-002 를 사용한다.
embedding_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
)

# 문서 -> 벡터 변환
vectorstore = Chroma.from_documents(pages, embedding_model) # 숫자, 벡터 형태
#리트리버가 유사한 문서를 찾아서 벡터가 아닌 문자 형태로 변황!
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [27]:
# 5. OpenAI LLM(Chat Model 연결)
# 언어 모델은 gpt-35-turbo , gpt-35-turbo-instruct 둘 중 하나를 사용하거나 두
# 개 모두 사용할 수 있다.
# 최대 출력 토큰 수는 512 이하로 설정한다.

from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    max_tokens=512, 
    temperature=0.2,
)

In [28]:
# 6. Chain 연결 without LCEL

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False)

In [29]:
# 7. [구현 사항] 아래 3개 질문에 대한 LLM 모델의 답 3개를 JSON 형태로 출력한다.

# * query1 = "파일의 주요 내용을 1줄로 요약해줘"
# * query2 = "2021년 기준 20대의 1일 평균 엽산 섭취량을 알려줘"
# * query3 = "질병관리청에서 추천하는 비타민12가 많이 들어있는 음식 3가지를 알려줘"

query1 = {"query" : "파일의 주요 내용을 1줄로 요약해줘"}
query2 = {"query" : "2021년 기준 20대의 1일 평균 엽산 섭취량을 알려줘"}
query3 = {"query" : "질병관리청에서 추천하는 비타민12가 많이 들어있는 음식 3가지를 알려줘"}

queries = [query1, query2, query3]

In [ ]:
# 8. Custom Prompt

from langchain.prompts import PromptTemplate

# [기능 요구사항]
# 사용자 질문에 대해 짧고 간략한 답변을 제시한다.
# 문서에 나와있지 않은 내용에 대해서는 명시되어있지 않다고 답변해야한다.
# 모든 답변은 “질병관리청에서 알려드립니다.”로 시작한다.

custom_template = '''사용자 질문에 대해 짧고 간략한 답변을 제시한다.
문서에 나와있지 않은 내용에 대해서는 명시되어있지 않다고 답변해야한다.
모든 답변은 “질병관리청에서 알려드립니다.”로 시작한다.

* 자료 : {context}
* 사용자 질문 : {question}
'''
custom_prompt = PromptTemplate(
    template=custom_template,
    input_variables=[
        'context',
        'question',
    ]
)

new_chain = RetrievalQA.from_chain_type(llm=chat_llm,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        return_source_documents=False,
                                        chain_type_kwargs={"prompt": custom_prompt})

# 주어지는 사용자 질문(query)에 대한 결과를 ai_response 로 반환한다.
ai_response = new_chain.batch(queries)


In [34]:
# 아래 형식에 맞춰 결과를 출력해주세요
# key 값이 다음과 완전히 일치해야 채점이 진행됩니다 : name, team , ID_number,
# ai_response1, ai_response2, ai_response3

final_result = {
"name": "좌경윤",
"team" : "System개발팀",
"ID_number": "97158424",
"ai_response1": ai_response[0]["result"],
"ai_response2": ai_response[1]["result"],
"ai_response3": ai_response[2]["result"]
}

# JSON으로 출력
import json
print(json.dumps(final_result, ensure_ascii=False))

{"name": "좌경윤", "team": "System개발팀", "ID_number": "97158424", "ai_response1": "질병관리청에서 실시하는 국민건강영양조사에서는 국민의 건강·영양 수준을 파악하고 건강증진 및 만성질환 예방관리 정책 효과 모니터링 지표를 산출하며, 이에 대한 연구 결과 중 하나는 2013-2015 국민건강영양조사에서 수집된 혈청에서 엽산, 비타민 B12, 호모시스테인 농도를 측정하여 분포 및 적정 상태를 평가한 것이다.", "ai_response2": "질병관리청에서 알려드립니다. 2021년 기준 20대의 1일 평균 엽산 섭취량은 259.3 (㎍ DFE)입니다.", "ai_response3": "질병관리청에서 알려드립니다. 높은 비타민 B12 함량을 가진 음식으로는 소고기, 연어, 훈제연어 등이 있습니다. 하지만, 비타민 B12 섭취량이 부족한 경우에는 보충제를 복용하는 것이 좋습니다."}
